In [1]:
import numpy as np

In [2]:
def createDataSet():
    group = np.array([[1, 1.1], [1, 1], [0,0], [0,0.1]])
    labels = ['A', 'A', 'B', 'B']
    return group , labels

In [3]:
###%save kNN.py 1-2

In [4]:
import kNN

In [5]:
group , labels = kNN.createDataSet()

np.tile作用

In [6]:
np.tile([0,0],5)###1方向重复5次

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [7]:
np.tile([0,0],(3,2))###行3 列2

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

np.argsort()

In [8]:
?np.argsort()

In [9]:
np.argsort([5,3,4])###从左到右 返回的是从小到大值的index

array([1, 2, 0], dtype=int64)

In [10]:
group

array([[ 1. ,  1.1],
       [ 1. ,  1. ],
       [ 0. ,  0. ],
       [ 0. ,  0.1]])

In [11]:
labels

['A', 'A', 'B', 'B']

In [12]:
inx=np.array([1,1.5])

In [13]:
diffmat=np.tile(inx,(group.shape[0],1))-group

In [14]:
diffmat

array([[ 0. ,  0.4],
       [ 0. ,  0.5],
       [ 1. ,  1.5],
       [ 1. ,  1.4]])

In [15]:
sqdiffmat=diffmat**2

In [16]:
sqdiffmat

array([[ 0.  ,  0.16],
       [ 0.  ,  0.25],
       [ 1.  ,  2.25],
       [ 1.  ,  1.96]])

In [17]:
sqdiffmat=sqdiffmat.sum(1)

In [18]:
sqdiffmat

array([ 0.16,  0.25,  3.25,  2.96])

In [19]:
distance=sqdiffmat**0.5

In [20]:
distance

array([ 0.4       ,  0.5       ,  1.80277564,  1.72046505])

In [21]:
order=distance.argsort()

In [22]:
from collections import defaultdict

In [23]:
dict1 = defaultdict(lambda:0)

In [24]:
k=3

In [25]:
for i in range(k):
    dict1[labels[order[i]]]+=1
    

In [26]:
dict1

defaultdict(<function __main__.<lambda>>, {'A': 2, 'B': 1})

In [27]:
dict2={'a':13,'d':31,'df':3}

In [28]:
sorted(dict2.items(),key=lambda x:x[1])

[('df', 3), ('a', 13), ('d', 31)]

In [29]:
import operator

In [30]:
sorted(dict2.items(), key = operator.itemgetter(1))

[('df', 3), ('a', 13), ('d', 31)]

In [31]:
sorted(dict1.items(),key=lambda x:x[1],reverse=True)[0][0]

'A'

In [32]:
def classify0(inx,dataset,labels,k):
    diffmat = np.tile(inx,(dataset.shape[0],1)) - dataset
    square_diff=diffmat**2
    sum_diff=square_diff.sum(1)
    distances=sum_diff**0.5
    order_index = distances.argsort()
    dict1=defaultdict(lambda:0)
    for i in range(k):
        dict1[labels[order_index[i]]]+=1
    sort_label=sorted(dict1.items(),key=lambda x:x[1],reverse=True)
    return sort_label[0][0]

In [33]:
pwd()

'C:\\Users\\49928\\Desktop\\Machine-Learning-in-Action\\CH 2'

In [34]:
import os

In [35]:
os.chdir(r'C:\Users\49928\Desktop\Machine-Learning-in-Action')

In [36]:
with open('./resources/machinelearninginaction/Ch02/datingTestSet.txt') as file:
    a = [line.strip() for i,line in enumerate(file) ]

In [37]:
def file2matrix(filename):
    classLabel = []
    returnmat = np.zeros((1000,3))
    index = 0
    with open(filename,'r') as file1:
        for line in file1:
            aa = line.strip().split('\t')
            list_line = aa[:3]
            list_line = list(map(lambda x:float(x), list_line))
            returnmat[index, :] = list_line
            classLabel.append(aa[-1])
            index +=1
    return returnmat, classLabel

In [38]:
a,b=file2matrix('./resources/machinelearninginaction/Ch02/datingTestSet.txt')

In [39]:
np.unique(b)

array(['didntLike', 'largeDoses', 'smallDoses'],
      dtype='<U10')

In [40]:
dict_1 ={}
for i,name in enumerate(np.unique(b)):
    dict_1[name] = i+1
dict_1

{'didntLike': 1, 'largeDoses': 2, 'smallDoses': 3}

In [41]:
for i,index in enumerate(b):
    b[i] = dict_1[index]

In [42]:
def autoNorm(dataSet):
    minvals = dataSet.min(0)
    maxvals = dataSet.max(0)
    ranges = maxvals - minvals
    normDataset = (dataSet - minvals)/ranges
    return normDataset, ranges, minvals

In [43]:
def datingclasstest(k, ratio = 0.1):
    data_all, labels = file2matrix('./resources/machinelearninginaction/Ch02/datingTestSet.txt')
    normmat, ranges, minvals = autoNorm(data_all)
    m = normmat.shape[0]
    Ind = np.arange(m)
    np.random.shuffle(Ind)
    test_size = int(m*ratio)
    test_data = normmat[Ind[:test_size],:]
    train_data = normmat[Ind[test_size:],:]
    label_train = [labels[i] for i in Ind[test_size:]]
    label_test = [labels[i] for i in Ind[:test_size]]
    error_count = 0
    for i in range(test_size):
        class_result = classify0(test_data[i,:],train_data,label_train,k)
        #print(f'the classifier came back with :{class_result},the real one is{label_test[i]}')
        if class_result != label_test[i]:
            error_count +=1
    return error_count/test_size

In [44]:
datingclasstest(6)

0.03

In [45]:
def classifyperson():
    a = float(input('percentage of time spent playing video game'))
    b = float(input('frequent filer miles earned per year?'))
    c = float(input('literrs of ice cream consumed per year'))
    data_all, labels = file2matrix('./resources/machinelearninginaction/Ch02/datingTestSet.txt')
    normmat, ranges, minvals = autoNorm(data_all)
    input_array = np.array([a,b,c])
    results = classify0((input_array - minvals)/ranges, normmat, labels , 6)
    print(results)
    

In [46]:
classifyperson()

percentage of time spent playing video game1
frequent filer miles earned per year?1
literrs of ice cream consumed per year1
smallDoses


In [59]:
def img2vector(filename):
    returnvect = np.zeros((1,1024))
    with open(filename) as fr:
        for i in range(32):
            linestr = fr.readline()
            for j in range(32):
                returnvect[0, 32*i + j] = int(linestr[j])
    return returnvect 

In [60]:
import os

In [61]:
os.listdir('./resources/machinelearninginaction/Ch02/trainingDigits/')

['0_0.txt',
 '0_1.txt',
 '0_10.txt',
 '0_100.txt',
 '0_101.txt',
 '0_102.txt',
 '0_103.txt',
 '0_104.txt',
 '0_105.txt',
 '0_106.txt',
 '0_107.txt',
 '0_108.txt',
 '0_109.txt',
 '0_11.txt',
 '0_110.txt',
 '0_111.txt',
 '0_112.txt',
 '0_113.txt',
 '0_114.txt',
 '0_115.txt',
 '0_116.txt',
 '0_117.txt',
 '0_118.txt',
 '0_119.txt',
 '0_12.txt',
 '0_120.txt',
 '0_121.txt',
 '0_122.txt',
 '0_123.txt',
 '0_124.txt',
 '0_125.txt',
 '0_126.txt',
 '0_127.txt',
 '0_128.txt',
 '0_129.txt',
 '0_13.txt',
 '0_130.txt',
 '0_131.txt',
 '0_132.txt',
 '0_133.txt',
 '0_134.txt',
 '0_135.txt',
 '0_136.txt',
 '0_137.txt',
 '0_138.txt',
 '0_139.txt',
 '0_14.txt',
 '0_140.txt',
 '0_141.txt',
 '0_142.txt',
 '0_143.txt',
 '0_144.txt',
 '0_145.txt',
 '0_146.txt',
 '0_147.txt',
 '0_148.txt',
 '0_149.txt',
 '0_15.txt',
 '0_150.txt',
 '0_151.txt',
 '0_152.txt',
 '0_153.txt',
 '0_154.txt',
 '0_155.txt',
 '0_156.txt',
 '0_157.txt',
 '0_158.txt',
 '0_159.txt',
 '0_16.txt',
 '0_160.txt',
 '0_161.txt',
 '0_162.txt',
 '0

In [62]:
def handwritingclasstest():
    labels=[]
    traininglist = os.listdir('./resources/machinelearninginaction/Ch02/trainingDigits/')
    m = len(traininglist)
    trainmat = np.zeros((m, 1024))
    j = 0
    for i in traininglist:
        str1 = i.split('_')[0]
        labels.append(str1)
        trainmat[j,:] = img2vector('./resources/machinelearninginaction/Ch02/trainingDigits/' + i)
        j +=1
    testlist = os.listdir('./resources/machinelearninginaction/Ch02/trainingDigits/')
    error_count = 0
    num = 1
    for a in testlist:
        vectortest = img2vector('./resources/machinelearninginaction/Ch02/trainingDigits/' + a)
        true_re = a.split('_')[0]
        classifyresult = classify0(vectortest, trainmat, labels, 3)
        num += 1
        if classifyresult != true_re:
            error_count +=1
    return(error_count/num)

In [64]:
handwritingclasstest()

0.011886304909560724